In [1]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt

from numpy.random import randn
from numpy.matlib import repmat

from scipy.stats import norm
from scipy.optimize import fmin
from scipy.special import erf

from patsy import dmatrices

from sklearn import metrics
from sklearn.linear_model import LogisticRegressionCV, LinearRegression, LassoCV
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MultiLabelBinarizer
from sklearn.feature_selection import chi2
from sklearn.ensemble import GradientBoostingClassifier

In [2]:
a = pd.DataFrame([[np.nan, 2, 3], [4, np.nan, 6]])

a[0] = a[0].fillna(0)

In [3]:
def array_vector(col):
    return np.array(str(col))

arrayerize = np.vectorize(array_vector)

def one_hot_encode(df, column, labels_column=None, whitelist=[]):
    # This is gross but since strings are iterable, we have to wrap them in a list
    # in order for the binarizer to parse the labels as strings and not chars
    labels = arrayerize(pd.DataFrame(df[column]))
    terms = arrayerize(pd.DataFrame(list(set(df[column]))))

    mlb = MultiLabelBinarizer()
    mlb.fit(terms)
    mlb.transform(labels)
    columns = [ f'{column}-{classname}' for classname in  mlb.classes_]

    encoded = pd.DataFrame(mlb.transform(labels), columns=columns, index=df[column].index)

    df.drop(column, axis=1, inplace=True)
    return df.join(encoded)

In [4]:
firms = pd.read_csv('../match-data/match-v1/firms.csv')
jobs = pd.read_csv('../match-data/match-v1/job-openings.csv')
jobs_parent = pd.read_csv('../match-data/match-v1/job-openings-parent.csv')
job_seekers = pd.read_csv('../match-data/match-v1/job-seekers.csv')
matches = pd.read_csv('../match-data/match-v1/matches.csv')
matches_parent = pd.read_csv('../match-data/match-v1/matches-parent.csv')


matches_merged = pd.merge(matches, matches_parent, on='number')
jobs_merged = pd.merge(jobs, jobs_parent, on='number')

In [5]:
categorical_columns = ["gender", "highest_edu_level", "nationality", "gendermix_not_allowed", "benefit1", "benefit2", "city", "english_proficiency", "impairments", "major", "opposite_gender_coworkers", "opposite_gender_manager", "first_job_field_preference", "second_job_field_preference", "bus_covered", "childcare_subsidy_offered", "dorm_covered", "driving_ability_required", "education_required", "english_proficiency_required", "female_requied", "free_meals_at_wok", "health_insurance_offered", "hearing_disability_accepted", "housing_subsidy_offered", "it_proficiency_required", "job_category", "job_description", "job_production", "jordanian_experience_required", "literacy_required", "male_required", "meal_subsidy_offered", "night_shifts_required", "noncognitive_skill_preference1", "noncognitive_skill_preference2", "numeracy_requied", "physical_disability_accepted", "physical_work_abilities_required", "problem_solving_required", "school_subsidy_offered", "specialization_required", "speech_disability_accepted", "syrian_considered", "transport_subsidy_offered", "visual_disability_accepted", "work_permit_offered",]
all_columns = ["age", "gender", "highest_edu_level", "will_work_night_shift", "nationality", "gendermix_not_allowed", "will_work_qiz", "arab_coworkers", "benefit1", "benefit2", "city", "daily_hours_willing_to_work", "days_willing_train_unpaid", "distance_willing_to_travel", "english_proficiency", "experience_clerical_work", "experience_factory", "experience_management_work", "experience_manual_labor", "experience_professional_work", "follow_up_agreement", "has_job", "impairments", "major", "nonarab_coworkers", "opposite_gender_coworkers", "opposite_gender_manager", "weekly_days_willing_to_work", "will_live_in_dorm", "will_train_unpaid", "years_education", "years_exp", "first_job_field_preference", "rwage1", "second_job_field_preference", "hh_income", "hired_yes_no", "quit", "fired", "interest_applying", "num_children", "personal_income", "bus_covered", "childcare_subsidy_offered", "dorm_covered", "driving_ability_required", "education_required", "english_proficiency_required", "female_requied", "free_meals_at_wok", "health_insurance_offered", "hearing_disability_accepted", "housing_subsidy_offered", "it_proficiency_required", "job_category", "job_description", "job_production", "jordanian_experience_required", "literacy_required", "male_required", "meal_subsidy_offered", "night_shifts_required", "noncognitive_skill_preference1", "noncognitive_skill_preference2", "num_vacancies", "numeracy_requied", "physical_disability_accepted", "physical_work_abilities_required", "problem_solving_required", "school_subsidy_offered", "specialization_required", "speech_disability_accepted", "syrian_considered", "transport_subsidy_offered", "visual_disability_accepted", "wage_offered", "work_permit_offered", "years_experience_required",]
scalar_columns = ["age", "daily_hours_willing_to_work", "days_willing_train_unpaid", "distance_willing_to_travel", "years_education", "years_exp", "rwage1", "hh_income", "num_children", "personal_income", "num_vacancies", "wage_offered"]

In [6]:
job_seekers['parent_case_id'] = job_seekers['caseid']

In [7]:
merged = pd.merge(job_seekers, matches_merged, on='parent_case_id')
merged = pd.merge(merged, jobs_merged, on='job_id')

merged.to_csv('../match-data/match-v1/merged.csv')

In [8]:
formatted = pd.DataFrame()
for col in all_columns:
    formatted[col] = merged[col]

dvs = ['hired_yes_no', 'quit', 'fired']

for col in all_columns:
    if col not in scalar_columns and col not in dvs:
        formatted = one_hot_encode(formatted, col)

for col in formatted.columns:
    # TODO: Change this
    formatted[col] = formatted[col].replace(['---'], 0)
    
formatted.to_csv('../match-data/match-v1/formatted.csv')

In [9]:
formatted['hired_yes_no'] = formatted['hired_yes_no'].fillna(0)
formatted['hired_yes_no'] = formatted['hired_yes_no'].replace(['---'], 0)

formatted['quit'] = formatted['quit'].fillna(0)
formatted['quit'] = formatted['quit'].replace(['---'], 0)
formatted['quit'] = formatted['quit'].replace(['no'], 0)
formatted['quit'] = formatted['quit'].replace(['yes'], 1)

formatted['fired'] = formatted['fired'].fillna(0)
formatted['fired'] = formatted['fired'].replace(['---'], 0)
formatted['fired'] = formatted['fired'].replace(['no'], 0)
formatted['fired'] = formatted['fired'].replace(['yes'], 1)

formatted['hired_yes_no'] = formatted['hired_yes_no'].astype(bool)
formatted['quit'] = formatted['quit'].astype(bool)
formatted['fired'] = formatted['fired'].astype(bool)

y = pd.DataFrame()
y['retained'] = formatted['hired_yes_no'] & ~(formatted['quit'] | formatted['fired'])
#y['hired'] = formatted['hired_yes_no']

In [10]:
formatted = formatted.drop(dvs, axis=1)

In [11]:
to_drop = []
for col in formatted.columns:
    if col in scalar_columns:
        formatted[col] = formatted[col].astype(float)
        mean = formatted[col].mean()
        formatted[col] = formatted[col].replace(['---'], mean)
        formatted[col] = formatted[col].fillna(mean)
    else:
        formatted[col] = formatted[col].fillna(0)
        formatted[col] = formatted[col].astype(int)
        formatted[col] = formatted[col].replace(['---'], 0)
    
    if col.endswith('---'):
        to_drop.append(col)

formatted = formatted.drop(columns=to_drop)

In [12]:
y=y.astype('int')
y.to_csv('../match-data/match-v1/y.csv')

In [13]:
formatted.to_csv('../match-data/match-v1/X.csv')
X = formatted

In [14]:
print(X.shape, y.shape)

(112, 245) (112, 1)


## Random Forest

In [15]:
from sklearn.ensemble import RandomForestClassifier
scores = []
model = RandomForestClassifier(max_depth=10, random_state=0, n_estimators=100)

for i in range(100):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
    model.fit(X_train, y_train)
    s = model.score(X_test, y_test)
    scores.append(s)
    
sum(scores) / len(scores)

0.7575675675675677

In [23]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
model.fit(X_train, y_train)
# Extract single tree
estimator = model.estimators_[5]

from sklearn.tree import export_graphviz
# Export as dot file
export_graphviz(estimator, out_file='tree.dot', 
                feature_names = X.columns,
                class_names = [str(c) for c in model.classes_],
                rounded = True, proportion = False, 
                precision = 2, filled = True)

# Convert to png using system command (requires Graphviz)
from subprocess import call
call(['dot', '-Tpng', 'tree.dot', '-o', 'tree.png', '-Gdpi=600'])

# Display in jupyter notebook
from IPython.display import Image
Image(filename = 'tree.png')

In [ ]:
importances = {}
for i, importance in enumerate(regr.feature_importances_):
    importances[X.columns[i]] = importance
    
importances

c = {}
for k, v in importances.items():
    c[k] = [v]
    

importance_frame = pd.DataFrame.from_dict(c)
importance_frame.to_csv('../match-data/match-v1/random-forest-importance.csv')

# GBRT

In [ ]:
scores = []
for i in range(100):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
    model = GradientBoostingClassifier()
    model.fit(X_train, y_train)
    scores.append(model.score(X_test, y_test))

sum(scores) / len(scores)

## LASSO CV

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
reg = LassoCV(cv=3, normalize=True).fit(X_train, y_train)
reg.score(X_test, y_test)

# K Best Feature Elimination

In [ ]:
from sklearn.feature_selection import SelectKBest

# Create and fit selector
selector = SelectKBest(chi2, k=10)
selector.fit(X, y)
# Get idxs of columns to keep
idxs_selected = selector.get_support(indices=True)
cols = []
for i, c in enumerate(X.columns):
    if i in idxs_selected:
        cols.append(c)

### Accuracy using top 10 k-best features

In [ ]:
model = LogisticRegressionCV(max_iter=1000)

scores = []
for i in range(1000):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
    XX_train = pd.DataFrame()
    XX_test = pd.DataFrame()
    for c in cols:
        XX_train[c] = X_train[c]
        XX_test[c] = X_test[c]

    model.fit(XX_train, y_train)
    scores.append(model.score(XX_test, y_test))

sum(scores) / len(scores)

### Export Coefs & p-values to csv

In [ ]:
coef_dict = {}
for coef, feat in zip(model.coef_[0], cols):
    coef_dict[feat] = coef
    
c = {}
for k, v in coef_dict.items():
    c[k] = [v]
    

coef_frame = pd.DataFrame.from_dict(c)
sorted_frame = coef_frame.columns[coef_frame.ix[coef_frame.last_valid_index()].argsort()]
coef_frame.to_csv('../match-data/match-v1/coefs.csv')

odds_ratios = np.exp(coef_frame)
scores, pvalues = chi2(X, y)

p_dict = {}
for pvalue, feat in zip(pvalues, cols):
    p_dict[feat] = pvalue
    
c = {}
for k, v in p_dict.items():
    c[k] = [v]
    

pval_frame = pd.DataFrame.from_dict(c)
sorted_frame = pval_frame.columns[pval_frame.ix[pval_frame.last_valid_index()].argsort()]
pval_frame.to_csv('../match-data/match-v1/pvalues.csv')

In [ ]:
display(coef_frame)
display(pval_frame)
display(odds_ratios)

In [ ]:
combined = coef_frame.append(pval_frame).append(np.exp(odds_ratios))
combined.to_csv('../match-data/match-v1/combined.csv')

# Recursive Feature Elimination

In [ ]:
model = LogisticRegressionCV(max_iter=1000, multi_class='multinomial', solver='lbfgs')
from sklearn.feature_selection import RFE
selector = RFE(model, 10, step=1)
selector.fit(X, y)

In [ ]:
len(selector.support_)
features = []
for i, v in enumerate(selector.support_):
    if v:
        features.append(X.columns[i])

features

### Accuracy using top 10 RFE features

In [ ]:
model = LogisticRegressionCV(max_iter=1000, multi_class='multinomial', solver='lbfgs')
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
XX = pd.DataFrame()
for c in features:
    XX[c] = X[c]

model.fit(XX, y)
model.score(XX, y)